# Use Different Combinations of SageMaker Components

1. [Example 1: build, train, and deploy _**all**_ on SageMaker](#Example1)

2. [Example 2: bring own training script](#Example2)

3. [Example 3: bring own trained model](#Example3)

4. [Example 4: bring own container](#Example4)

<a id="Example1"></a>
# Example 1: build, train, and deploy _**all**_ on SageMaker

---
1. Introduction
2. Prerequisites and Preprocessing
    1. Data ingestion
    2. Data inspection
    3. Data conversion
    4. Data uploading
3. Training
4. Hosting
5. Validating
6. (Optional) Clean-up
---

## Introduction

Using Amazon SageMaker's Built-in Linear Learner to Predict Whether a Handwritten Digit is a 0.

> Amazon SageMaker's Linear Learner algorithm extends upon typical linear models by training many models in parallel, in a computationally efficient manner. Each model has a different set of hyperparameters, and then the algorithm finds the set that optimizes a specific criteria. This can provide substantially more accurate models than typical linear algorithms at the same, or lower, cost.

## Prerequisites and Preprocessing

Kernel in SageMaker Studio: `Data Science`

In [ ]:
import sagemaker

role = sagemaker.get_execution_role()
bucket = sagemaker.Session().default_bucket()  # “sagemaker-{region}-{aws-account-id}”
prefix = 'demo-linear-mnist'

### Data ingestion

Next, we read the dataset into memory, for preprocessing prior to training. This processing could be done *in situ* by Amazon Athena, Apache Spark in Amazon EMR, Amazon Redshift, etc., assuming the dataset is present in the appropriate location. Then, the next step would be to transfer the data to S3 for use in training. For small datasets, such as this one, reading into memory isn't onerous, though it would be for larger datasets.

In [ ]:
%%time
import pickle, gzip

# Manually download mnist.pkl.gz from https://www.kaggle.com/pablotab/mnistpklgz
# Upload mnist.pkl.gz to ./datasets

# Load the dataset
with gzip.open('./datasets/mnist.pkl.gz', 'rb') as f:
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')

### Data inspection

Once the dataset is imported, it's typical as part of the machine learning process to inspect the data, understand the distributions, and determine what type(s) of preprocessing might be needed. You can perform those tasks right here in the notebook. As an example, let's go ahead and look at one of the digits that is part of the dataset.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (2, 10)


def show_digit(img, caption='', subplot=None):
    if subplot == None:
        _, (subplot) = plt.subplots(1, 1)
    imgr = img.reshape((28, 28))
    subplot.axis('off')
    subplot.imshow(imgr, cmap='gray')
    plt.title(caption)

show_digit(train_set[0][30], 'This is a {}'.format(train_set[1][30]))

### Data conversion

Since algorithms have particular input and output requirements, converting the dataset is also part of the process that a data scientist goes through prior to initiating training. In this particular case, the Amazon SageMaker implementation of Linear Learner takes recordIO-wrapped protobuf, where the data we have today is a pickle-ized numpy array on disk.

In [ ]:
import io
import numpy as np
import sagemaker.amazon.common as smac

vectors = np.array([t.tolist() for t in train_set[0]]).astype('float32')
labels = np.where(np.array([t.tolist() for t in train_set[1]]) == 0, 1, 0).astype('float32')

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, vectors, labels)
buf.seek(0)

### Data uploading

Now that we've created our recordIO-wrapped protobuf, we'll need to upload it to S3, so that Amazon SageMaker training can use it.

In [ ]:
import boto3
import os

key = 'recordio-pb-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train', key)).upload_fileobj(buf)
s3url_train_data = 's3://{}/{}/train/{}'.format(bucket, prefix, key)
print(s3url_train_data)

Let's also setup an output S3 location for the model artifact that will be output as the result of training with the algorithm.

In [ ]:
s3url_output = 's3://{}/{}/output'.format(bucket, prefix)
print(s3url_output)

## Training

First let's specify our algorithm image. 

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

image = get_image_uri(boto3.Session().region_name, 'linear-learner')

Next we'll kick off the base estimator, making sure to pass in the necessary hyperparameters.  Notice:
- `feature_dim` is set to 784, which is the number of pixels in each 28 x 28 image.
- `predictor_type` is set to 'binary_classifier' since we are trying to predict whether the image is or is not a 0.
- `mini_batch_size` is set to 200.  This value can be tuned for relatively minor improvements in fit and speed, but selecting a reasonable value relative to the dataset is appropriate in most cases.

In [ ]:
import boto3

linear = sagemaker.estimator.Estimator(image,
                                       role, 
                                       train_instance_count=1, 
                                       train_instance_type='ml.m5.xlarge',
                                       output_path=s3url_output,
                                       sagemaker_session=sagemaker.Session())

linear.set_hyperparameters(feature_dim=784,
                           predictor_type='binary_classifier',
                           mini_batch_size=200)

linear.fit({'train': s3url_train_data})

## Hosting

Now that we've trained our model, we can deploy it behind an Amazon SageMaker real-time hosted endpoint.  This will allow us to make predictions (or inference) from the model dyanamically. (_Note, Amazon SageMaker allows you the flexibility of importing models trained elsewhere, as well as the choice of not importing models if the target of model creation is AWS Lambda, AWS Greengrass, Amazon Redshift, Amazon Athena, or other deployment target._)

In [ ]:
# (Optional) Use an existing model trained by SageMaker

job_name = 'linear-learner-2021-02-20-17-21-37-544'
linear = sagemaker.estimator.Estimator.attach(job_name)

In [ ]:
all_instance_types = ["ml.r5d.12xlarge", "ml.r5.12xlarge", "ml.p2.xlarge", "ml.m5.4xlarge", "ml.m4.16xlarge", "ml.r5d.24xlarge", "ml.r5.24xlarge", "ml.p3.16xlarge", "ml.m5d.xlarge", "ml.m5.large", "ml.t2.xlarge", "ml.p2.16xlarge", "ml.m5d.12xlarge", "ml.inf1.2xlarge", "ml.m5d.24xlarge", "ml.c4.2xlarge", "ml.c5.2xlarge", "ml.c4.4xlarge", "ml.inf1.6xlarge", "ml.c5d.2xlarge", "ml.c5.4xlarge", "ml.g4dn.xlarge", "ml.g4dn.12xlarge", "ml.c5d.4xlarge", "ml.g4dn.2xlarge", "ml.c4.8xlarge", "ml.c4.large", "ml.c5d.xlarge", "ml.c5.large", "ml.g4dn.4xlarge", "ml.c5.9xlarge", "ml.g4dn.16xlarge", "ml.c5d.large", "ml.c5.xlarge", "ml.c5d.9xlarge", "ml.c4.xlarge", "ml.inf1.xlarge", "ml.g4dn.8xlarge", "ml.inf1.24xlarge", "ml.m5d.2xlarge", "ml.t2.2xlarge", "ml.c5d.18xlarge", "ml.m5d.4xlarge", "ml.t2.medium", "ml.c5.18xlarge", "ml.r5d.2xlarge", "ml.r5.2xlarge", "ml.p3.2xlarge", "ml.m5d.large", "ml.m5.xlarge", "ml.m4.10xlarge", "ml.t2.large", "ml.r5d.4xlarge", "ml.r5.4xlarge", "ml.m5.12xlarge", "ml.m4.xlarge", "ml.m5.24xlarge", "ml.m4.2xlarge", "ml.p2.8xlarge", "ml.m5.2xlarge", "ml.r5d.xlarge", "ml.r5d.large", "ml.r5.xlarge", "ml.r5.large", "ml.p3.8xlarge", "ml.m4.4xlarge"]
for i in range(len(all_instance_types)):
    try:
        linear_predictor = linear.deploy(initial_instance_count=1,
                                         instance_type=all_instance_types[i])
    except:
        pass
    else:
        print('\nUsing instance type: ', all_instance_types[i])
        break

## Validating

Finally, we can now validate the model for use.  We can pass HTTP POST requests to the endpoint to get back predictions.  We'll specify how to serialize requests and deserialize responses that are specific to the algorithm.

In [ ]:
from sagemaker.predictor import csv_serializer, json_deserializer

# linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = csv_serializer
linear_predictor.deserializer = json_deserializer

Now let's try to get a prediction for a single record.

In [ ]:
result = linear_predictor.predict(train_set[0][30:31])
print(result)

We should see that for one record our endpoint returned some JSON which contains `predictions`, including the `score` and `predicted_label`.  In this case, `score` will be a continuous value between [0, 1] representing the probability we think the digit is a 0 or not.  `predicted_label` will take a value of either `0` or `1` where (somewhat counterintuitively) `1` denotes that we predict the image is a 0, while `0` denotes that we are predicting the image is not of a 0.

Let's do a whole batch of images and evaluate our predictive accuracy.

In [ ]:
import numpy as np

predictions = []
for array in np.array_split(test_set[0], 100):
    result = linear_predictor.predict(array)
    predictions += [r['predicted_label'] for r in result['predictions']]

predictions = np.array(predictions)

In [ ]:
import pandas as pd
pd.crosstab(np.where(test_set[1] == 0, 1, 0), predictions, rownames=['actuals'], colnames=['predictions'])

## (Optional) Clean-up

To avoid incurring unnecessary charges, delete the endpoints and resources that you created while running this example.

Delete endpoints

In [ ]:
sagemaker.Session().delete_endpoint(linear_predictor.endpoint)

Delete models

In [ ]:
import boto3
from pprint import pprint

client = boto3.client('sagemaker')


def main():
    model_names = []
    for key in paginate(client.list_models):
        model_names.append(key['ModelName'])
    delete_multiple_models(model_names)


def delete_multiple_models(model_names):
    for model_name in model_names:
        print('Deleting model: {}'.format(model_name))
        client.delete_model(ModelName=model_name)


def paginate(method, **kwargs):
    client = method.__self__
    paginator = client.get_paginator(method.__name__)
    for page in paginator.paginate(**kwargs).result_key_iters():
        for result in page:
            yield result

            
main()

Delete S3 bucket

Open the Amazon S3 [console](https://console.aws.amazon.com/s3/), and then delete the bucket that you created for storing model artifacts and the training dataset.

Delete logs

Open the Amazon CloudWatch [console](https://console.aws.amazon.com/cloudwatch/), and then delete all of the log groups that have names starting with `/aws/sagemaker/`.

<a id="Example2"></a>
# Example 2: bring own training script

<a id="Example3"></a>
# Example 3: bring own trained model

<a id="Example4"></a>
# Example 4: bring own container